In [1]:
import pandas as pd

df = pd.read_csv('../data/metpolice/2018-07-metropolitan-street.csv').dropna(subset=['Longitude', 'Latitude'])

print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94577 entries, 0 to 94576
Data columns (total 12 columns):
Crime ID                 73388 non-null object
Month                    94577 non-null object
Reported by              94577 non-null object
Falls within             94577 non-null object
Longitude                94577 non-null float64
Latitude                 94577 non-null float64
Location                 94577 non-null object
LSOA code                94577 non-null object
LSOA name                94577 non-null object
Crime type               94577 non-null object
Last outcome category    73388 non-null object
Context                  0 non-null float64
dtypes: float64(3), object(9)
memory usage: 9.4+ MB
None


In [2]:
df['Coordinate'] = df.apply(lambda x: (x['Longitude'], x['Latitude']), axis=1)

df.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,Coordinate
0,e9fe81ec7a6f5d2a80445f04be3d7e92831dbf3090744e...,2018-07,Metropolitan Police Service,Metropolitan Police Service,0.774271,51.148147,On or near Bethersden Road,E01024031,Ashford 012B,Other theft,Under investigation,NaN,"(0.774271, 51.148146999999994)"
1,076b796ba1e1ba3f69c9144e2aa7a7bc85b61d51bf7a59...,2018-07,Metropolitan Police Service,Metropolitan Police Service,-1.007293,51.893136,On or near Prison,E01017674,Aylesbury Vale 010D,Other crime,Awaiting court outcome,NaN,"(-1.007293, 51.893136)"
2,163e996d58995cf87d14f15711fbd87052681919f02029...,2018-07,Metropolitan Police Service,Metropolitan Police Service,0.744706,52.038219,On or near Hillside Road,E01029918,Babergh 007F,Violence and sexual offences,Under investigation,NaN,"(0.7447060000000001, 52.038219)"
3,NaN,2018-07,Metropolitan Police Service,Metropolitan Police Service,0.137065,51.583672,On or near Police Station,E01000027,Barking and Dagenham 001A,Anti-social behaviour,NaN,NaN,"(0.13706500000000002, 51.583672)"
4,NaN,2018-07,Metropolitan Police Service,Metropolitan Police Service,0.148434,51.595164,On or near Park/Open Space,E01000027,Barking and Dagenham 001A,Anti-social behaviour,NaN,NaN,"(0.14843399999999998, 51.595164000000004)"


In [3]:
crime_sr = df.groupby(['Coordinate', 'Crime type']).size()

In [4]:
features = []

last_coord = None
data = None

for (coord, crime_type), crime_count in crime_sr.iteritems():
    
    # Create a new data if it's the new location
    if coord != last_coord:
        if data is not None:
            features.append(data)
                
        # Re-initialise the data
        data = {
            'type': 'Feature',
            'properties': {
                'totalCrimeCount': 0,
                'crimeCountPerType': {}
            },
            'geometry': {
                'type': 'Point',
                'coordinates': []
            }
        }
        
    data['properties']['totalCrimeCount'] += crime_count
    data['properties']['crimeCountPerType'][crime_type] = crime_count
    data['geometry']['coordinates'] = [coord[0], coord[1]]
    
    last_coord = coord

In [5]:
import json

with open('./template.geojson', 'r') as template:
    template = json.load(template)
    template['features'] = features

with open('improve_data_aggeregation.geojson', 'w') as output:
    json.dump(template, output)